In [1]:
import pandas as pd #Basic packages for creating dataframes and loading dataset
import numpy as np

import matplotlib.pyplot as plt #Package for visualization

import re #importing package for Regular expression operations

from sklearn.model_selection import train_test_split #Package for splitting the data

from sklearn.preprocessing import LabelEncoder #Package for conversion of categorical to Numerical

from keras.preprocessing.text import Tokenizer #Tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences #Add zeros or crop based on the length
from keras.models import Sequential #Sequential Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #For layers in Neural Network
from keras.utils.np_utils import to_categorical

2024-10-24 18:11:27.484372: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = pd.read_csv("Sentiment.csv")

In [32]:
import pandas as pd

# Load the dataset as a Pandas DataFrame
dataset = pd.read_csv('Sentiment.csv', header=0)

# Select only the necessary columns 'text' and 'sentiment'
mask = dataset.columns.isin(['text', 'sentiment'])
#print(data.columns)
data = dataset.loc[:, mask]

In [33]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

/var/folders/2c/271qr1m16ss8v45r4_tfpn700000gn/T/ipykernel_13140/2214349272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())
/var/folders/2c/271qr1m16ss8v45r4_tfpn700000gn/T/ipykernel_13140/2214349272.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))


In [34]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') #Removing Retweets

/var/folders/2c/271qr1m16ss8v45r4_tfpn700000gn/T/ipykernel_13140/1671198718.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt', ' ') #Removing Retweets
/var/folders/2c/271qr1m16ss8v45r4_tfpn700000gn/T/ipykernel_13140/1671198718.py:2: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt', ' ') #Removing Retweets


In [35]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ') #Maximum words is 2000 to tokenize sentence
tokenizer.fit_on_texts(data['text'].values) 
X = tokenizer.texts_to_sequences(data['text'].values) #taking values to feature matrix

In [36]:
X = pad_sequences(X) #Padding the feature matrix

embed_dim = 128 #Dimension of the Embedded layer
lstm_out = 196 #Long short-term memory (LSTM) layer neurons

In [37]:
def createmodel():
    model = Sequential() #Sequential Neural Network
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1])) #input dimension 2000 Neurons, output dimension 128 Neurons
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)) #Drop out 20%, 196 output Neurons, recurrent dropout 20%
    model.add(Dense(3,activation='softmax')) #3 output neurons[positive, Neutral, Negative], softmax as activation
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) #Compiling the model
    return model
# print(model.summary())


In [39]:
labelencoder = LabelEncoder() #Applying label Encoding on the label matrix
integer_encoded = labelencoder.fit_transform(data['sentiment']) #fitting the model
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) #67% training data, 33% test data split

In [40]:
batch_size = 32 #Batch size 32
model = createmodel() #Function call to Sequential Neural Network
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) #verbose the higher, the more messages
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size) #evaluating the model
print(score)
print(acc)

291/291 - 23s - loss: 0.8266 - accuracy: 0.6447 - 23s/epoch - 79ms/step
144/144 - 2s - loss: 0.7666 - accuracy: 0.6706 - 2s/epoch - 17ms/step
0.7666078805923462
0.6705985069274902


In [41]:
print(model.metrics_names) #metrics of the model

['loss', 'accuracy']


In [42]:
model.save('sentimentAnalysis.h5') #Saving the model

In [43]:
from keras.models import load_model #Importing the package for importing the saved model
model= load_model('sentimentAnalysis.h5') #loading the saved model

In [44]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [50]:
# Predicting on the text data
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence) # Tokenizing the sentence
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0) # Padding the sentence
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0] # Predicting the sentence text
sentiment = np.argmax(sentiment_probs)
print(sentiment_probs)
if sentiment >= 0:
    print("Positive")
elif sentiment < 0:
    print("Negative")
else:
    print("Cannot be determined")

1/1 - 0s - 21ms/epoch - 21ms/step
[0.6091014  0.20005693 0.19084163]
Positive


In [51]:
from keras.wrappers.scikit_learn import KerasClassifier #importing Keras classifier
from sklearn.model_selection import GridSearchCV #importing Grid search CV

model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters
batch_size= [10, 20, 40] #hyper parameter batch_size
epochs = [1, 2] #hyper parameter no. of epochs
param_grid= {'batch_size':batch_size, 'epochs':epochs} #creating dictionary for batch size, no. of epochs
grid  = GridSearchCV(estimator=model, param_grid=param_grid) #Applying dictionary with hyper parameters
grid_result= grid.fit(X_train,Y_train) #Fitting the model
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #best score, best hyper parameters

/var/folders/2c/271qr1m16ss8v45r4_tfpn700000gn/T/ipykernel_13140/140920398.py:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters


744/744 - 41s - loss: 0.8300 - accuracy: 0.6455 - 41s/epoch - 55ms/step
186/186 - 2s - loss: 0.7596 - accuracy: 0.6740 - 2s/epoch - 10ms/step
744/744 - 22s - loss: 0.8226 - accuracy: 0.6472 - 22s/epoch - 29ms/step
186/186 - 1s - loss: 0.7721 - accuracy: 0.6708 - 738ms/epoch - 4ms/step
744/744 - 15s - loss: 0.8229 - accuracy: 0.6450 - 15s/epoch - 20ms/step
186/186 - 1s - loss: 0.7747 - accuracy: 0.6703 - 727ms/epoch - 4ms/step
744/744 - 14s - loss: 0.8247 - accuracy: 0.6453 - 14s/epoch - 19ms/step
186/186 - 1s - loss: 0.7523 - accuracy: 0.6760 - 709ms/epoch - 4ms/step
744/744 - 15s - loss: 0.8194 - accuracy: 0.6410 - 15s/epoch - 20ms/step
186/186 - 1s - loss: 0.7708 - accuracy: 0.6701 - 738ms/epoch - 4ms/step
Epoch 1/2
744/744 - 15s - loss: 0.8218 - accuracy: 0.6453 - 15s/epoch - 21ms/step
Epoch 2/2
744/744 - 14s - loss: 0.6757 - accuracy: 0.7168 - 14s/epoch - 18ms/step
186/186 - 1s - loss: 0.7439 - accuracy: 0.6832 - 719ms/epoch - 4ms/step
Epoch 1/2
744/744 - 14s - loss: 0.8156 - accur